<a href="https://colab.research.google.com/github/amitupreti/sentiment-analysis-from-scratch-With-Numpy/blob/master/Part_1_sentiment_analysis_with_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Reading the reviews and Label

In [1]:
# mounting and navigating to google drive
from google.colab import drive
drive.mount('gdrive')
%cd gdrive/'My Drive'/'sentiment-analysis-from-scratch-With-Numpy'

# comment the box before commiting to github

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive
/content/gdrive/My Drive/sentiment-analysis-from-scratch-With-Numpy


In [2]:
with open('reviews.txt','r') as f:
    reviews_raw = f.read().splitlines()
with open('labels.txt','r') as f:
    labels_raw = f.read().splitlines()

print('------'*2+'Review1'+'-----'*2)
print(reviews_raw[0])

print('\n'+'------'*2+'Label1'+'-----'*2)
print(labels_raw[0])

------------Review1----------
bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   

------------Label1----------
positive


In [0]:
#### A function to preety print review and label

def pretty_print_review_and_label(index):
    print(f'{labels_raw[index]}\t:\t{reviews_raw[index][:80]} ...')

In [4]:
pretty_print_review_and_label(1)
pretty_print_review_and_label(100)
pretty_print_review_and_label(1401)
pretty_print_review_and_label(10000)

negative	:	story of a man who has unnatural feelings for a pig . starts out with a opening  ...
positive	:	i find it so amazing that even after all these years  we are still talking about ...
negative	:	when i was   i saw the documentary  a funny thing happened on the way to the moo ...
positive	:	another one for the babes  bullets crowd . the story is much edgier than any oth ...


### 1.1 Develop A predictive Theory

###### Here we are supposed to check the dataset and try to figure out what makes the review postive or negative. Looking at few samples an idea that seems likely is that positive reviews  have more number of positive words such as "great","amazing","loved" and negative reviews have more number of negative words such as "hate","terrible"



### 1.2 Validating our Theory

##### Here we will create two dictionary which will contain the frequency of each word in positive and negative review
1. positive_counts 
2. negatice_counts

also a third dictionary(total_counts) to hold count of all the words


In [0]:
from collections import Counter
import numpy as np

In [0]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

for i,curr_label in enumerate(labels_raw):
    curr_review = reviews_raw[i]
    for word in curr_review.split(" "):
        if curr_label == 'positive':
            positive_counts[word] += 1
        elif curr_label == 'negative':
            negative_counts[word] += 1
        total_counts[word] += 1
    
        

Lets view the most common words in our postive and negative reviews.m

In [7]:
positive_counts.most_common(20)

[('', 550468),
 ('the', 173324),
 ('.', 159654),
 ('and', 89722),
 ('a', 83688),
 ('of', 76855),
 ('to', 66746),
 ('is', 57245),
 ('in', 50215),
 ('br', 49235),
 ('it', 48025),
 ('i', 40743),
 ('that', 35630),
 ('this', 35080),
 ('s', 33815),
 ('as', 26308),
 ('with', 23247),
 ('for', 22416),
 ('was', 21917),
 ('film', 20937)]

In [8]:
negative_counts.most_common(20)

[('', 561462),
 ('.', 167538),
 ('the', 163389),
 ('a', 79321),
 ('and', 74385),
 ('of', 69009),
 ('to', 68974),
 ('br', 52637),
 ('is', 50083),
 ('it', 48327),
 ('i', 46880),
 ('in', 43753),
 ('this', 40920),
 ('that', 37615),
 ('s', 31546),
 ('was', 26291),
 ('movie', 24965),
 ('for', 21927),
 ('but', 21781),
 ('with', 20878)]

In [0]:
## we can see that the most repeated words are "the", ".", "to"  etc 
#which might not best reflect the sentiment of the review

## so now we will try to find words that appear more often on positive reviews than negative reviews and vice versa
#to accomplish this, you'll need to calculate the ratios of word usage between positive and negative reviews.

In [0]:
pos_neg_ratios = Counter()
for word,count in total_counts.items():
    if count >100: # considering words that are at least repeated 100 times
        pos_neg_ratios[word] = positive_counts[word]/(1+negative_counts[word])

In [0]:
# pos_neg_ratios

In [12]:
print(f'Positive to negative ratio for word "the" {pos_neg_ratios["the"]}')
print(f'Positive to negative ratio for word "amazing" {pos_neg_ratios["amazing"]}')
print(f'Positive to negative ratio for word "terrible" {pos_neg_ratios["terrible"]}')

Positive to negative ratio for word "the" 1.0607993145235326
Positive to negative ratio for word "amazing" 4.022813688212928
Positive to negative ratio for word "terrible" 0.17744252873563218


In [0]:
## we can see that the words which we expect to be positive have higher  positive to negative values than 1 and the words which we expect
## to be negative have values close to 0
## and the words that mostly donot convey sentiments have value close to 1

## This kind of validates our theory

### this positive to negative ratio can be an estimate for us to tell if the word is positive or negative

In [14]:
pos_neg_ratios['bad']

0.2576330721426641

In [15]:
pos_neg_ratios['love']

1.9976776590803529

Looking closely at the values you just calculated, we see the following:

* Words that you would expect to see more often in positive reviews – like "amazing" – have a ratio greater than 1. The more skewed a word is toward postive, the farther from 1 its positive-to-negative ratio will be.
* Words that you would expect to see more often in negative reviews – like "terrible" – have positive values that are less than 1. The more skewed a word is toward negative, the closer to zero its positive-to-negative ratio will be.
* Neutral words, which don't really convey any sentiment because you would expect to see them in all sorts of reviews – like "the" – have values very close to 1. A perfectly neutral word – one that was used in exactly the same number of positive reviews as negative reviews – would be almost exactly 1. The +1 we suggested you add to the denominator slightly biases words toward negative, but it won't matter because it will be a tiny bias and later we'll be ignoring words that are too close to neutral anyway.

Ok, the ratios tell us which words are used more often in postive or negative reviews, but the specific values we've calculated are a bit difficult to work with. A very positive word like "amazing" has a value above 4, whereas a very negative word like "terrible" has a value around 0.18. Those values aren't easy to compare for a couple of reasons:

* Right now, 1 is considered neutral, but the absolute value of the postive-to-negative rations of very postive words is larger than the absolute value of the ratios for the very negative words. So there is no way to directly compare two numbers and see if one word conveys the same magnitude of positive sentiment as another word conveys negative sentiment. So we should center all the values around netural so the absolute value fro neutral of the postive-to-negative ratio for a word would indicate how much sentiment (positive or negative) that word conveys.
* When comparing absolute values it's easier to do that around zero than one.* 

In [0]:
## Now we will convert the rations to logarithm so that the values can be centered around zero

In [0]:
for word,value in pos_neg_ratios.most_common():
    pos_neg_ratios[word] = np.log(value)

In [18]:
print(f'Positive to negative ratio for word "the" {pos_neg_ratios["the"]}')
print(f'Positive to negative ratio for word "amazing" {pos_neg_ratios["amazing"]}')
print(f'Positive to negative ratio for word "terrible" {pos_neg_ratios["terrible"]}')

Positive to negative ratio for word "the" 0.05902269426102881
Positive to negative ratio for word "amazing" 1.3919815802404802
Positive to negative ratio for word "terrible" -1.7291085042663878


In [0]:
## Here we can see neutral words have value close to zero
## negative words have negative values
## and postitive words have postivive values

In [20]:
pos_neg_ratios.most_common(30) ## top 30 positive words

[('edie', 4.6913478822291435),
 ('paulie', 4.07753744390572),
 ('felix', 3.152736022363656),
 ('polanski', 2.8233610476132043),
 ('matthau', 2.80672172860924),
 ('victoria', 2.681021528714291),
 ('mildred', 2.6026896854443837),
 ('gandhi', 2.538973871058276),
 ('flawless', 2.451005098112319),
 ('superbly', 2.26002547857525),
 ('perfection', 2.159484249353372),
 ('astaire', 2.1400661634962708),
 ('captures', 2.038619547159581),
 ('voight', 2.030170492673053),
 ('wonderfully', 2.0218960560332353),
 ('powell', 1.978345424808467),
 ('brosnan', 1.9547990964725592),
 ('lily', 1.9203768470501485),
 ('bakshi', 1.9029851043382795),
 ('lincoln', 1.9014583864844796),
 ('refreshing', 1.8551812956655511),
 ('breathtaking', 1.8481124057791867),
 ('bourne', 1.8478489358790986),
 ('lemmon', 1.8458266904983307),
 ('delightful', 1.8002701588959635),
 ('flynn', 1.7996646487351682),
 ('andrews', 1.7764919970972666),
 ('homer', 1.7692866133759964),
 ('beautifully', 1.7626953362841438),
 ('soccer', 1.757857

In [21]:
pos_neg_ratios.most_common()[:-31:-1] # 30 most negative words

[('boll', -4.969813299576001),
 ('uwe', -4.624972813284271),
 ('seagal', -3.644143560272545),
 ('unwatchable', -3.258096538021482),
 ('stinker', -3.2088254890146994),
 ('mst', -2.9502698994772336),
 ('incoherent', -2.9368917735310576),
 ('unfunny', -2.6922395950755678),
 ('waste', -2.6193845640165536),
 ('blah', -2.5704288232261625),
 ('horrid', -2.4849066497880004),
 ('pointless', -2.4553061800117097),
 ('atrocious', -2.4259083090260445),
 ('redeeming', -2.3682390632154826),
 ('prom', -2.3608540011180215),
 ('drivel', -2.3470368555648795),
 ('lousy', -2.307572634505085),
 ('worst', -2.286987896180378),
 ('laughable', -2.264363880173848),
 ('awful', -2.227194247027435),
 ('poorly', -2.2207550747464135),
 ('wasting', -2.204604684633842),
 ('remotely', -2.1972245773362196),
 ('existent', -2.0794415416798357),
 ('boredom', -1.995100393246085),
 ('miserably', -1.9924301646902063),
 ('sucks', -1.987068221548821),
 ('uninspired', -1.9832976811269336),
 ('lame', -1.981767458946166),
 ('insult

## 2. Transforming Text into Numbers

We know that neural networks are just matrix multiplications and we can't perform these matrix multiplications on text data . so now we will have to convert the text input to numerical input

We will create a big 1d array of for all the words that we have from both positive and negative reviews.

so for each input, we will assign 1 to if that word is found in our big 1d array and leave others as zero

Also, for output we will use 0 to indicate negative sentiment and 1 to indicate positive sentiment

In [23]:
from IPython.display import Image

review = "This was a horrible, terrible movie."

Image(filename='network.png')

FileNotFoundError: ignored

In [24]:
review = "The movie was excellent"

Image(filename='sentiment_network_pos.png')

FileNotFoundError: ignored

In [29]:
# llets create a set called vocab that lists all the words
vocab = set(total_counts.keys())
vocab_size = len(vocab)
vocab_size

74074

In [30]:
from IPython.display import Image
Image(filename='sentiment_network_2.png')

#Take a look at the following image. It represents the layers of the neural network you'll be building 
#throughout this notebook. layer_0 is the input layer, layer_1 is a hidden layer, 
#and layer_2 is the output layer.



FileNotFoundError: ignored

In [32]:
# defining the first input layer for our neural network. this is of the size of our all the words in our data corpus
layer_0 = np.zeros((1,vocab_size))
layer_0.shape


(1, 74074)

In [34]:
from IPython.display import Image
Image(filename='sentiment_network.png')

FileNotFoundError: ignored

In [0]:
#now we will create a lookup dictionary that maps all the text in our data corpus to their position, so that we can lookup the index of every word
word2index = {}
for i,word in enumerate(vocab):
  word2index[word] = i



In [57]:
# viewing a slice of our word to index mapping
# https://stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
{k: word2index[k] for k in list(word2index)[:10]}

{'': 0,
 'aji': 9,
 'antheil': 2,
 'cherishing': 5,
 'jekyll': 4,
 'melnik': 7,
 'sangster': 8,
 'shtoop': 1,
 'submarines': 6,
 'visitor': 3}

In [0]:
# update_input_layer function
'''

TODO: Complete the implementation of update_input_layer. It should count how many times each word is used in the given review, and then store those counts at the appropriate indices inside layer_0.
'''
def update_input_layer(review):

  """
  Modifies the global layer_0 to represent the vector form of  review.
  The element at a given index of layer_0 represents
  how many times the word has occured in the review.

  Args:
       review(string) -  the string of the review
  Returns:
       None
  """
  # why using global function not local function??/?
  global layer_0
  layer_0 *= 0 
  for word in review.split(' '):
    layer_0[0][word2index[word]] += 1

In [63]:
update_input_layer(reviews_raw[0])
layer_0

array([[18.,  0.,  0., ...,  0.,  0.,  0.]])

In [0]:
def get_target_for_label(label):
  """
  Converrts a label to 0 or 1
  Args:
    label(string) - "POSITIVE" or NEGATIVE
  Returns:
     0 or 1
  """
  label = label.upper()
  if label == 'POSITIVE':
    return 1
  else:
    return 0

In [69]:
labels_raw[0]



'positive'

In [70]:
get_target_for_label(labels_raw[0])

1